# Drugi domaci zadatak - Katarina Petrovic, 2021/68

In [ ]:
import skimage
from skimage import color
from skimage import exposure
from skimage import filters
from skimage import io
from skimage import util
from skimage import restoration

import numpy as np
from scipy import ndimage
from skimage.util import view_as_windows

import time
import matplotlib.pyplot as plt

### Prvi zadatak - Polutoniranje

In [ ]:
#Notch filtar
def notch(filt_type, notch, Ny, Nx, C, sigma_x, sigma_y, n=1):

    N_filters = len(C)
    filter_mask = np.zeros([Ny,Nx])

    #Postavljanje koordinata slike tako da je (0,0) u centru slike
    if (Ny%2 == 0):
        y = np.arange(0,Ny) - Ny/2 + 0.5
    else:
        y = np.arange(0,Ny) - (Ny-1)/2
  
    if (Nx%2 == 0):
        x = np.arange(0,Nx) - Nx/2 + 0.5
    else:
        x = np.arange(0,Nx) - (Nx-1)/2
   
    #Mreza slike
    X, Y = np.meshgrid(x, y)
   
    #Primenjujemo filtriranje oko svakog centra iz C
    for i in range(0, N_filters):
        C_current = C[i]
        
        #Kreiranje komplementarnog centra
        C_complement = np.zeros([2,1])
        C_complement[0] = -C_current[0]
        C_complement[1] = -C_current[1]

        #Kreiramo x0, y0 za rastojanje D0
        if (Ny%2 == 0):
            y0 = y - C_current[0] + Ny/2 - 0.5
        else:
            y0 = y - C_current[0] + (Ny-1)/2
        
        if (Nx%2 == 0):
            x0 = x - C_current[1] + Nx/2 - 0.5
        else:
            x0 = x - C_current[1] + (Nx-1)/2
 
        X0, Y0 = np.meshgrid(x0, y0)
        D0 = np.sqrt(np.square(X0) + np.square(Y0))

        #Kreiramo x0c, y0c za komplementarne centre i njihovo D0c
        if (Ny%2 == 0):
            y0c = y - C_complement[0] - Ny/2 + 0.5
        else:
            y0c = y - C_complement[0] - (Ny-1)/2
        
        if (Nx%2 == 0):
            x0c = x - C_complement[1] - Nx/2 + 0.5
        else:
            x0c = x - C_complement[1] - (Nx-1)/2

        X0c, Y0c = np.meshgrid(x0c, y0c)
        D0c = np.sqrt(np.square(X0c) + np.square(Y0c))
 
        #Na osnovu izbora filtra radi se neko od filtriranja - Gauss-ov filtar, idealni filtar ili Butterworth-ov filtar
        if filt_type == 'gaussian':
            #Na masku koju cine nule dodajemo Gausovu funkciju varijanse r
            filter_mask = filter_mask + np.exp(-((X0**2) / (2 * sigma_x**2) + (Y0**2) / (2 * sigma_y**2))) + np.exp(-((X0c**2) / (2 * sigma_x**2) + (Y0c**2) / (2 * sigma_y**2)))
        elif filt_type == 'btw':
            #Na masku koju cine nule dodajemo fju btw filtra zavisno od reda filtra i \"varijanse\" r
            filter_mask = filter_mask + 1/(1+(D0/sigma_x)**(2*n)) + 1/(1+(D0c/sigma_x)**(2*n))
        elif filt_type == 'ideal':
            #Na masku koju cine nule dodajemo fju idealnog filtra - unutar poluprecnika r su jedinice, van su 0
            filter_mask[(D0<=sigma_x)|(D0c<=sigma_x)] = 1
        else:
            print('Greška! Nije podržan tip filtra: ', filt_type)
            return
  
    #Biramo pass ili reject funkcionalnost zavisno sta zelimo da ostavimo na slici
    if notch == 'pass':
        return filter_mask
    elif notch == 'reject':
        return 1 - filter_mask
    else:
        return

In [ ]:
#Funkcija low pass filtra
#Parametri sigma_x i sigma_y se koriste kod Gausove funkcije ukoliko zelimo razlicite varijanse po x i y pravcima
#U ostalim tipovima filtra, sigma_y nam ne treba
def lpfilter(filt_type, Ny, Nx, sigma_x, sigma_y, n=1):

    #Postavljanje koordinata slike tako da je (0,0) u centru slike
    if (Ny%2 == 0):
        y = np.arange(0,Ny) - Ny/2 + 0.5
    else:
        y = np.arange(0,Ny) - (Ny-1)/2
    
    if (Nx%2 == 0):
        x = np.arange(0,Nx) - Nx/2 + 0.5
    else:
        x = np.arange(0,Nx) - (Nx-1)/2
    
    
    X, Y = np.meshgrid(x, y)
    
    #D je rastojanje svake tacke (koordinate X i Y) od centra slike
    D = np.sqrt(np.square(X) + np.square(Y))
    
    #Na osnovu tipa filtra kreiramo filter masku
    if filt_type == 'gaussian':
        #Gausova filter maska, moze imati razlicitu varijansu po x i y pravcima
        filter_mask = np.exp(-((X**2) / (2 * sigma_x**2) + (Y**2) / (2 * sigma_y**2)))
    elif filt_type == 'btw':
        #Maska Batervort-ovog filtra
        filter_mask = 1/(1+(D/sigma_x)**(2*n))
    elif filt_type == 'ideal':
        #Idealni filtar
        filter_mask = np.ones([Ny, Nx])
        filter_mask[D>sigma_x] = 0
    else:
        print('Greška! Nije podržan tip filtra: ', filt_type)
        return
    
    return filter_mask

In [ ]:
img = util.img_as_float(io.imread('../sekvence/girl_ht.tif'))
    
#Prikaz ulazne slike
fig, ax = plt.subplots(2,2, figsize=(12,6))
ax[0, 0].imshow(img, cmap='gray')
ax[0, 0].set_title('Ulazna slika')
ax[0, 0].axis('off')

#Dimenzije slike
[Ny, Nx] = np.shape(img)

#Frekvencijski domen slike
img_fft = np.fft.fftshift(np.fft.fft2(img))

#Prikaz FFT ulazne slike
ax[0, 1].imshow(np.log(1+abs(img_fft)), cmap='gray')
ax[0, 1].set_title('Ulazna slika u frekvencijskom domenu')
ax[0, 1].axis('off')

h_lp = lpfilter('gaussian', Ny, Nx, 120, 120)
img_fft = img_fft*h_lp

#U FFT se pojavljuje 16 tacaka (sumova) po horizontali i vertikali
broj_tacaka = 16
#Koraci za iteraciju i postavljanje filtara
korak_x = int(Nx/broj_tacaka)
korak_y = int(Ny/broj_tacaka)

#U niz centara (C) ubacujemo pozicije sumova - prvih 7 kolona i prvih 7 elemenata 8-e kolone 
#Ostali ce se simetricno generisati u funkciji - filtar mora biti centralno simetrican
C = []
for i in range(1,int(broj_tacaka/2)):
    for j in range(1, broj_tacaka):
        C.append([j*korak_y,i*korak_x])
for i in range(1,int(broj_tacaka/2)):
    C.append([i*korak_y,int(broj_tacaka/2*korak_x)])

#Primena Cnotch filtra
#Frekvencijski domen Cnotch filtra sa varijansom 20 - ispostavlja se kao odgovarajuca vrednost
nr_filter_freq = notch('gaussian', 'reject', Ny, Nx, C, 30, 25)

#Prikaz filtra
ax[1, 0].imshow(np.log(1+abs(nr_filter_freq)), cmap='gray')
ax[1, 0].set_title('Notch filtar u frekvencijskom domenu')
ax[1, 0].axis('off')


#Slika nakon filtriranja Cnotch filtrom
img_fft_filt = img_fft*nr_filter_freq

#Prikaz frekvencijskog domena izlazne slike
ax[1, 1].imshow(np.log(1+abs(img_fft_filt)), cmap='gray')
ax[1, 1].set_title('Izlazna slika u frekvencijskom domenu')
ax[1, 1].axis('off')


#Vracamo se u prostorni domen nakon filtriranja
#Ocekujemo samo realne vrednosti jer je frekvencijski domen simetrican
img_filt = np.real(np.fft.ifft2(np.fft.ifftshift(img_fft_filt)))

#Radi sigurnosti zelimo sve vrednosti da ogranicimo na opseg 0-1 (fiksiramo > na 1 i < na 0)
img_filt = np.clip(img_filt, 0, 1)

#Sliku vracamo u opseg 0-255
img_filt = (img_filt * 255).astype(np.uint8)

#Uporedni prikaz ulazne i izlazne slike
fig, ax = plt.subplots(1, 2, figsize=(12,8))

ax[0].imshow(img, cmap='gray')
ax[0].set_title('Ulazna slika')
ax[0].axis('off')

ax[1].imshow(img_filt, cmap='gray')
ax[1].set_title('Izlazna slika')
ax[1].axis('off')

### Drugi zadatak - restauracija

In [ ]:
#Ucitavanje slike
img = io.imread('../sekvence/road_blur.png')

#Razdvajanje na RGB komponente
img_r = img[:,:,0]
img_g = img[:,:,1]
img_b = img[:,:,2]

#Skaliranje slike na opseg 0-255
img_r = util.img_as_float(img_r)
img_g = util.img_as_float(img_g)
img_b = util.img_as_float(img_b)

[Ny, Nx] = np.shape(img_r)

#Generisanje Gausove raspodele - degradacione funkcije
filter_frekv = lpfilter('gaussian', Ny, Nx, 60, 45)

#Parametri Vinerovog filtra
k = 0.0002
W = (abs(filter_frekv)**2)/(abs(filter_frekv)**2 + k)


rgb = [img_r, img_g, img_b]
rgb_out = []
#U petlji generisemo restaurirane komponente RGB 
for i in range(3):
    img_i = rgb[i]
    #Spektar komponente slike - FFT
    img_fft = np.fft.fftshift(np.fft.fft2(img_i))
    #Spektar nakon deljenja degradacionom funkcijom
    img_fft_est = (img_fft/filter_frekv)
    #Spektar nakon primene Vinerovog filtra
    img_fft_est_filt = (img_fft/filter_frekv)*W
    #Vracanje u prostorni domen - inverzna FFT
    img_est = np.real(np.fft.ifft2(np.fft.ifftshift(img_fft_est_filt)))
    img_est = np.clip(img_est, 0, 1)
    

    rgb_out.append(img_est)

#Kombinovanje RGB komponenti u izlaznu sliku
img_out = np.stack((rgb_out[0], rgb_out[1], rgb_out[2]), axis=-1)

#Prikaz komponenti slike
fig, ax = plt.subplots(1,3, figsize=(12,10))
ax[0].imshow(rgb_out[0], cmap='gray')
ax[0].set_title('Izlazna slika - R komponenta')
ax[0].axis('off')

ax[1].imshow(rgb_out[1], cmap='gray')
ax[1].set_title('Izlazna slika - G komponenta')
ax[1].axis('off')


ax[2].imshow(rgb_out[2], cmap='gray')
ax[2].set_title('Izlazna slika - B komponenta')
ax[2].axis('off')

#Prikaz postupka restauracije - uradjen na B komponenti s obzirom da je ona poslednja obradjena u for petlji pa su te promenljive sacuvane
fig, ax = plt.subplots(2,2, figsize=(12,6))

ax[0, 0].imshow(np.log(1+abs(img_fft)), cmap='gray')
ax[0, 0].set_title('Spektar ulazne slike')
ax[0, 0].axis('off')

ax[0, 1].imshow(np.log(1+abs(filter_frekv)), cmap='gray')
ax[0, 1].set_title('Gausov filtar u frekvencijskom domenu')
ax[0, 1].axis('off')

ax[1, 0].imshow(np.log(1+abs(img_fft_est)), cmap='gray')
ax[1, 0].set_title('Spektar slike nakon deljenja Gausovim filtrom')
ax[1, 0].axis('off')

ax[1, 1].imshow(np.log(1+abs(img_fft_est_filt)), cmap='gray')
ax[1, 1].set_title('Spektar slike nakon Wiener-ovog filtra')
ax[1, 1].axis('off')

#Uporedni prikaz ulazne i izlazne slike
plt.figure(figsize=(10, 6))
plt.imshow(img, cmap='gray')
plt.title('Ulazna slika')
plt.axis('off')
plt.show()

# Druga figura za izlaznu sliku
plt.figure(figsize=(10, 6))
plt.imshow(img_out, cmap='gray')
plt.title('Izlazna slika')
plt.axis('off') 
plt.show()

### Treci zadatak - nelokalno usrednjavanje

In [ ]:
def dos_non_local_means(I, K, S, var, h):
    
    #Prozor dimenzije KxK (lokalno susedstvo piksela) ima radius K//2
    radius_mask = K//2
    
    #Norma je broj elemenata u masci
    norma1 = K**2
    
    #Prozor u okviru kog trazimo piksele za uporedjivanje ima radius S//2
    radius_win = S//2

    #Dimenzije ulazne slike
    M, N = np.shape(I)
    
    #Prosirivanje slike za vrednost radijusa maske sa svake strane
    padded_img = np.pad(I, pad_width=radius_mask, mode='reflect')

    #Prikaz svakog piksela kao njegovog lokalnog susedstva KxK,
    #windows je matrica dimenzija (M+r)*(N+r) gde je svaki element matrica KxK
    #Optimizacija funkcije da se ne bi svaki put u for petlji racunalo lokalno susedstvo piksela
    windows = view_as_windows(padded_img, (K,K))
    
    #Inicijalizacija izlaza
    J = np.zeros_like(I, dtype=float)
    
    #For petlja za iteraciju kroz sliku
    for m in range(M):
        for n in range(N):
            
            #Pozicija prozora u okviru kog trazimo piksele za uporedjivanje
            #Nece svaki prozor biti bas SxS da ne bi preterano prosirivali sliku
            #Piksele blizu ivica obradjujemo tako sto uzmemo sve piksele koji bi dolazili u obzir osim onih koji bi se samo zbog te potrebe prosirivali
            #Ovo optimizuje program, a i daje uverljivije rezultate - ne racunamo reflektovane piksele duplo
            pocetak_prozora_vrsta = max(0, m-radius_win)
            kraj_prozora_vrsta = min(m+radius_win+1, M)
            pocetak_prozora_kolona = max(0,n-radius_win)
            kraj_prozora_kolona = min(n+radius_win+1, N)
            
            #Prozor na slici I u kome trazimo piksele za uporedjivanje
            prozor_kratak = I[pocetak_prozora_vrsta:kraj_prozora_vrsta, pocetak_prozora_kolona:kraj_prozora_kolona]
        
            #Iste te koordinate u sistemu padded img - trebace za lokalno susedstvo svih tih piksela
            pocetak_prozora_vrsta_padded = pocetak_prozora_vrsta 
            kraj_prozora_vrsta_padded = kraj_prozora_vrsta + 2*radius_mask
            pocetak_prozora_kolona_padded = pocetak_prozora_kolona
            kraj_prozora_kolona_padded = kraj_prozora_kolona + 2*radius_mask
            
            #Prosireni prozor
            prozor = padded_img[pocetak_prozora_vrsta_padded:kraj_prozora_vrsta_padded, pocetak_prozora_kolona_padded:kraj_prozora_kolona_padded]
            
            #Sve maske unutar prozora, dimenzije KxK
            search_windows = np.lib.stride_tricks.sliding_window_view(prozor, (K, K))
            
            #Druga norma - prema definicji
            norma2 = np.sum((search_windows - windows[m,n]) ** 2, axis=(-2, -1))/norma1
            
            #Tezine
            tezine = np.exp(-np.maximum(norma2-2*sigma_n, 0)/h**2)
            
            #Izlaznoj slici dodeljujemo piksel na poziciji m, n sa sracunatom i normalizovanom vrednoscu
            J[m,n] = np.sum(tezine*prozor_kratak)/np.sum(tezine)
    return J

In [ ]:
#Priprema slike lena.tif za testiranje
img = io.imread('../sekvence/lena.tif')

#Sliku konvertujemo u opseg 0-1
img = img/255

#Dodavanje suma slici
sigma_n = 0.02
img_noise = util.random_noise(img, mode='gaussian', var=sigma_n)

In [ ]:
#Funkcija koja racuna PSNR izlazne slike funkcije dos_non_local_means i zasumljene slike
def calculate_psnr(img, img_out):  
    #Racunamo varijansu
    mse = np.mean((img_out - img) ** 2) 
    
    #Maksimalna varijansa za float sliku je 1
    max_var = 1
    
    #Racunamo PSNR po definiciji
    psnr = 10*np.log10((max_var / np.sqrt(mse)))
    
    return psnr

In [ ]:
#Testiramo za koje vrednosti h je PSNR maksimalno, fiksiramo vrednosti K=5 i S=15
psnr = []

#Ispitujemo opseg h od 0.01 do 0.3 (sa korakom 0.01)
test_vrednosti_h = np.arange(0.01, 0.3, 0.02)

for h in test_vrednosti_h:
    #Obradimo sliku funkcijom dos_non_local_means
    img_out = dos_non_local_means(img_noise, 5, 15, sigma_n, h)
    
    #Racunamo PSNR za izlaznu sliku
    psnr_h = calculate_psnr(img, img_out)
    psnr.append(psnr_h)

#Prikaz podataka, trazimo maksimum skupa psnr
plt.figure()
x_osa = np.arange(0.01, 0.3, 0.02)
plt.plot(x_osa, psnr)
plt.xlabel('h')
plt.ylabel('PSNR')
plt.title('Zavisnost PSNR od h')
plt.grid()
plt.show()

#Vrednost h za koju je PSNR maksimalno
h = x_osa[np.argmax(psnr)]
print(h)

In [ ]:
K = 5
S = 15

img_out = dos_non_local_means(img_noise, K, S, sigma_n, h)
psnr_out = calculate_psnr(img, img_out)
print(psnr_out)
fig, ax = plt.subplots(1,2, figsize=(12,8))
ax[0].imshow(img_noise, cmap='gray')
ax[0].set_title('Ulazna zasumljena slika')
ax[0].axis('off')

ax[1].imshow(img_out, cmap='gray')
ax[1].set_title('Izlazna slika')
ax[1].axis('off')

In [ ]:
#Formiramo matrice 3x3 - jednu u kojoj je svaki element slika sa zadatom vrednoscu K i S
#Drugu u kojoj je vrednost PSNR za tu kombinaciju K i S
#Ovo nam treba da bi prikazali kako se sa promenom parametara menjaju slika i PSNR
K_niz = np.array([3, 5, 9])
S_niz = np.array([15, 33, 51])

matrica_rezultujucih_slika = np.empty((3, 3), dtype=object)
matrica_psnr = np.empty((3, 3), dtype=float)
for i in range(len(K_niz)):
    for j in range(len(S_niz)):
        img_out = dos_non_local_means(img_noise, K_niz[i], S_niz[j], sigma_n, h)
        matrica_rezultujucih_slika[i,j] = img_out
        matrica_psnr[i,j] = calculate_psnr(img_out, img)
        


In [ ]:
#Prikaz slike i vrednost PSNR za razlicite K i S, h=0.08
fig, ax = plt.subplots(3,3, figsize=(12,12))
ax[0,0].imshow(matrica_rezultujucih_slika[0,0], cmap='gray', aspect='equal')
ax[0,0].set_title(f'K: 3, S = 15, PSNR: {matrica_psnr[0,0]:.2f}')
ax[0,0].axis('off')

ax[0,1].imshow(matrica_rezultujucih_slika[0,1], cmap='gray', aspect='equal')
ax[0,1].set_title(f'K: 3, S = 33, PSNR: {matrica_psnr[0,1]:.2f}')
ax[0,1].axis('off')

ax[0,2].imshow(matrica_rezultujucih_slika[0,2], cmap='gray', aspect='equal')
ax[0,2].set_title(f'K: 3, S = 51, PSNR: {matrica_psnr[0,2]:.2f}')
ax[0,2].axis('off')

ax[1,0].imshow(matrica_rezultujucih_slika[1,0], cmap='gray', aspect='equal')
ax[1,0].set_title(f'K: 5, S = 15, PSNR: {matrica_psnr[1,0]:.2f}')
ax[1,0].axis('off')

ax[1,1].imshow(matrica_rezultujucih_slika[1,1], cmap='gray', aspect='equal')
ax[1,1].set_title(f'K: 5, S = 33, PSNR: {matrica_psnr[1,1]:.2f}')
ax[1,1].axis('off')

ax[1,2].imshow(matrica_rezultujucih_slika[1,2], cmap='gray', aspect='equal')
ax[1,2].set_title(f'K: 5, S = 51, PSNR: {matrica_psnr[1,2]:.2f}')
ax[1,2].axis('off')

ax[2,0].imshow(matrica_rezultujucih_slika[2,0], cmap='gray', aspect='equal')
ax[2,0].set_title(f'K: 9, S = 15, PSNR: {matrica_psnr[2,0]:.2f}')
ax[2,0].axis('off')

ax[2,1].imshow(matrica_rezultujucih_slika[2,1], cmap='gray', aspect='equal')
ax[2,1].set_title(f'K: 9, S = 33, PSNR: {matrica_psnr[2,1]:.2f}')
ax[2,1].axis('off')

ax[2,2].imshow(matrica_rezultujucih_slika[2,2], cmap='gray', aspect='equal')
ax[2,2].set_title(f'K: 9, S = 51, PSNR: {matrica_psnr[2,2]:.2f}')
ax[2,2].axis('off')

In [ ]:
#Racunanje zavisnosti vremena i PSNR za razlicite vrednosti K i S
K_niz = np.array([3, 5, 9])
S_niz = np.arange(15, 52, 4)

#Inicijalizacija
matrica_psnr = np.empty((3, len(S_niz)), dtype=float)
matrica_vreme = np.empty((3, len(S_niz)), dtype=float)

for i in range(len(K_niz)):
    for j in range(len(S_niz)):
        start = time.time()
        img_out = dos_non_local_means(img_noise, K_niz[i], S_niz[j], sigma_n, h)
        end = time.time()
        matrica_vreme[i,j] = end - start
        matrica_psnr[i,j] = calculate_psnr(img_out, img)       


In [ ]:
#Prikaz grafika zavisnosti PSNR od S
plt.figure()
plt.plot(S_niz, matrica_psnr[0,:], label = 'K = 3')
plt.plot(S_niz, matrica_psnr[1,:], label = 'K = 5')
plt.plot(S_niz, matrica_psnr[2,:], label = 'K = 9')
plt.xlabel('S')
plt.ylabel('PSNR')
plt.title('Zavisnost PSNR od S')
plt.grid()
plt.legend()
plt.show()

In [ ]:
#Prikaz zavisnosti vremena izvrsavanja od S
plt.figure()
plt.plot(S_niz, matrica_vreme[0,:], label = 'K = 3')
plt.plot(S_niz, matrica_vreme[1,:], label = 'K = 5')
plt.plot(S_niz, matrica_vreme[2,:], label = 'K = 9')
plt.xlabel('S')
plt.ylabel('Vreme izvrsavanja')
plt.title('Zavisnost vremena izvrsavanja od S')
plt.grid()
plt.legend()
plt.show()